In [52]:
#!/usr/bin/env python
import rospy, sys
from std_msgs.msg import String
import tf
import numpy as np
import cv2

[x,y,z,w]=1,1,1,1
quaternion = (x,y,z,w)

euler = tf.transformations.euler_from_quaternion(quaternion)
quat = tf.transformations.quaternion_from_euler(1.57,0,1.57)
print quat
roll = euler[0]
pitch = euler[1]
yaw = euler[2]

print "Roll:  "+str(roll)
print "Pitch: "+str(pitch)
print "Yaw:   "+str(yaw)

[ 0.49999984  0.49960184  0.49999984  0.50039816]
Roll:  1.57079632679
Pitch: -0.0
Yaw:   1.57079632679


In [49]:
# ls /home/tushar/Downloads/Datasets/try_2/Yandex_Data/slam6DDat
points_3d = []
with open("/home/tushar/Downloads/Datasets/try_2/Yandex_Data/slam6DDat/scan000.3d") as scan:
    for line in scan:
        points_3d.append([float(i) for i in line.split()])

points_3d=np.array(points_3d)
points_3d = points_3d[:,:3]

In [50]:
points_3d.shape

(45117, 3)

In [36]:
image =  cv2.imread("/home/tushar/Downloads/Datasets/try_2/Yandex_Data/slam6DDat/scan000_center.jpeg")
len(image)
# from IPython.display import Image
# Image("/home/tushar/Downloads/Datasets/try_2/Yandex_Data/slam6DDat/scan000_center.jpeg")

1080

In [53]:
#camera
K = [1032.462842, 0.0, 653.03823, 0.0, 1019.154193, 565.644396, 0.0, 0.0, 1.0]

    

In [54]:
#transform velodyne_to_camera
translation = [-0.017559,0.033034,-0.298409]
rotation = [0.741396241526,0.0184214190141,0.0234637781524,-0.6704041434]


In [55]:
import math
rot = tf.transformations.quaternion_matrix(rotation)
# tf.transformations.projection_matrix([1,1,1],[0,0,1])
print rot
# tf.transformations.concatenate_matrices

[[ 0.9982202   0.05877557  0.01009232  0.        ]
 [-0.00414529 -0.10043787  0.9949347   0.        ]
 [ 0.05949151 -0.99320575 -0.10001547  0.        ]
 [ 0.          0.          0.          1.        ]]


In [56]:
tr = tf.transformations.translation_matrix(translation)
print tr

[[ 1.        0.        0.       -0.017559]
 [ 0.        1.        0.        0.033034]
 [ 0.        0.        1.       -0.298409]
 [ 0.        0.        0.        1.      ]]


In [57]:
#Return concatenation of series of transformation matrices.
Tr_vel_to_camera = tf.transformations.concatenate_matrices(rot,tr)
Tr_vel_to_camera

array([[ 0.9982202 ,  0.05877557,  0.01009232, -0.0185978 ],
       [-0.00414529, -0.10043787,  0.9949347 , -0.30014255],
       [ 0.05949151, -0.99320575, -0.10001547, -0.00400865],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [58]:
#camera_projection matrix
# print tf.transformations.projection_matrix([1,1,1],[1,2,1])
P =  np.array([[651.724159, 0.0,        622.685313, 0.0], 
               [0.0,        642.208128, 534.920382, 0.0],
               [0.0,        0.0,        1.0,        0.0]])


In [61]:
lidar_3d = np.array([[0],[0],[0],[1]])
im_projection = np.linalg.multi_dot([P,Tr_vel_to_camera,lidar_3d])
u = im_projection[0]/im_projection[2]
v = im_projection[1]/im_projection[2]


SyntaxError: EOL while scanning string literal (<ipython-input-65-ab0571e74431>, line 1)

In [18]:
points_3d

array([[  267.63 ,    28.615,   171.993],
       [  308.074,   138.498,   177.145],
       [  268.901,    36.389,   172.057],
       ..., 
       [ 3077.186,   701.491,   702.876],
       [  282.326,   136.061,   202.62 ],
       [ 2883.386,   739.784,   666.437]])

In [22]:
mapping_3d_2d = []
for i in points_3d:
    pose = np.array([i[0],i[1],i[2],1])
    im_projection = np.linalg.multi_dot([P,Tr_vel_to_camera,pose.transpose()])
    u = im_projection[0]/im_projection[2]
    v = im_projection[1]/im_projection[2]
    mapping_3d_2d.append([i,[u,v]])

In [25]:
np.savetxt("mapping_3d_2d.txt", mapping_3d_2d,fmt='%6s')

In [46]:
#filter points putside of image plane
mapping_3d_2d_filtered=[]
for i in mapping_3d_2d:
    if abs(i[1][0])<=1080 and abs(-i[1][1])<=1280:
        mapping_3d_2d_filtered.append(i)

In [47]:
np.savetxt("mapping_3d_2d_filtered.txt", mapping_3d_2d_filtered,fmt='%6s')

In [44]:
from math import *

In [45]:
abs(-1)

1